<a href="https://colab.research.google.com/github/IvanUlloa098/simulacion/blob/main/examen_final/EXAMEN_SIMPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import simpy
import random
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
LUGARES_DESTINO = ['GUAYAQUIL', 'QUITO', 'CUENCA', 'PORTOVIEJO','MACHALA', 'AMBATO']
PESOS_LUG_DES = (30, 20, 20, 15, 10, 5)
POBL_FEM = 0.53
PROB_MAS = 0.47
MEDIA_EDAD = 35
MEDIA_COMPRA = 10
DESVIACION_COMPRA = 2
MEDIA_VIAJE = 6 * 60
DESVIACION_VIAJE = 4 * 60
CAPACIDAD_BUS = 40
LLEGADA = 1
NUM_BUSES = 5

DIAS = 7
TIEMPO_SIMULACION = DIAS * 24 * 60

bernoulli = stats.bernoulli(POBL_FEM)
distribucion = bernoulli.rvs(100) # Universo arbitrario

In [ ]:
def escoger_lugar():
  return random.choices(LUGARES_DESTINO, weights=PESOS_LUG_DES, k=1)[0]

def generar_sexo():
    poblacion=stats.bernoulli(PROB_MAS)
    if poblacion.rvs(1)[0] == 1:
      return 'M'
    else:
      return 'F'

def generar_edad():
  return np.random.poisson(MEDIA_EDAD, 1)[0]

def tiempo_compra():
    return abs(random.normalvariate(MEDIA_COMPRA, DESVIACION_COMPRA))

def tiempo_viaje():
    return abs(random.normalvariate(MEDIA_VIAJE, DESVIACION_VIAJE))

def num_boletos():
  return random.randint(1,6)

In [ ]:
class AgenciaTurismo(object):
    
    def __init__(self, environment, nombre):
        self.env = environment
        self.bus = simpy.Resource(self.env, capacity=CAPACIDAD_BUS)
        self.nombre = nombre
        self.capacidad = CAPACIDAD_BUS
        self.sin_ticket = 0
    
    def comprar_ticket(self, cliente, tickets):
        yield self.env.timeout(tiempo_compra())
        if self.capacidad < tickets: 
          print(f'BOLETOS AGOTADOS para el cliente {cliente} compra {self.capacidad} boletos. TIEMPO={self.env.now}')
          tickets = self.capacidad
          self.capacidad = 0
          return tickets
        
        self.capacidad -= tickets
        print(f'Cliente {cliente} compra {tickets} boletos {self.env.now}')
        return tickets

    def ingreso_bus(self, cliente, ticket):
        with self.bus.request() as peticion:
          yield peticion   
          yield self.env.timeout(1)
          print(f'El pasajero con el ticket {ticket} comprado por {cliente} sube al transporte {self.nombre} TIEMPO={self.env.now}')
          self.bus.release(peticion)       

    def bus_deja_estacion(self):
        t = tiempo_viaje()
        yield self.env.timeout(t)
        print(f'El transporte {self.nombre} deja la estacion en {self.env.now} con {CAPACIDAD_BUS - self.capacidad} pasajeros')
        yield self.env.timeout(t)
        self.bus = simpy.Resource(self.env, capacity=CAPACIDAD_BUS)
        self.capacidad = CAPACIDAD_BUS
        print(f'El transporte {self.nombre} llega nuevamente a la estacion en el tiempo {self.env.now}')

In [ ]:
def run_agencia_turistica(env, nombre):
  agencia=AgenciaTurismo(env, nombre)
  i=0
  pasajeros=0
  tiempo_partida = 0
  
  while True:
      time_now = int(env.now/60)
      if (time_now == tiempo_partida):
        yield env.timeout(LLEGADA)
        cliente = {'id': f'cliente {i}', 'sexo':generar_sexo(), 'edad':f'{generar_edad()}','origen': escoger_lugar(), 'destino':escoger_lugar()}
        print(f'Llegada de {cliente} a la agencia TIEMPO={env.now}')
        res = yield env.process(agencia.comprar_ticket(cliente, num_boletos()))
        for j in range(res):
          pasajeros+=1
          yield env.process(agencia.ingreso_bus(cliente, j+1))

        i+=1
      else:
        tiempo_partida = time_now
        yield env.process(agencia.bus_deja_estacion())

In [ ]:
env = simpy.Environment()
env.process(run_agencia_turistica(env, 'bus 1'))
env.process(run_agencia_turistica(env, 'bus 2'))
env.process(run_agencia_turistica(env, 'bus 3'))
env.process(run_agencia_turistica(env, 'bus 4'))
env.process(run_agencia_turistica(env, 'bus 5'))

env.run(until = TIEMPO_SIMULACION)

Llegada de {'id': 'cliente 0', 'sexo': 'F', 'edad': '32', 'origen': 'CUENCA', 'destino': 'GUAYAQUIL'} a la agencia TIEMPO=1
Llegada de {'id': 'cliente 0', 'sexo': 'F', 'edad': '34', 'origen': 'GUAYAQUIL', 'destino': 'PORTOVIEJO'} a la agencia TIEMPO=1
Llegada de {'id': 'cliente 0', 'sexo': 'M', 'edad': '22', 'origen': 'QUITO', 'destino': 'QUITO'} a la agencia TIEMPO=1
Llegada de {'id': 'cliente 0', 'sexo': 'M', 'edad': '26', 'origen': 'GUAYAQUIL', 'destino': 'QUITO'} a la agencia TIEMPO=1
Llegada de {'id': 'cliente 0', 'sexo': 'M', 'edad': '21', 'origen': 'QUITO', 'destino': 'PORTOVIEJO'} a la agencia TIEMPO=1
Cliente {'id': 'cliente 0', 'sexo': 'M', 'edad': '21', 'origen': 'QUITO', 'destino': 'PORTOVIEJO'} compra 6 boletos 8.188479904313516
El pasajero con el ticket 1 comprado por {'id': 'cliente 0', 'sexo': 'M', 'edad': '21', 'origen': 'QUITO', 'destino': 'PORTOVIEJO'} sube al transporte bus 5 TIEMPO=9.188479904313516
El pasajero con el ticket 2 comprado por {'id': 'cliente 0', 'sexo